# Inkswarm DetectLab — Step Runner Notebook (D-0022)

This notebook lets you execute the pipeline **step-by-step** for **`login_attempt`**, while reusing the existing code structure.

## Steps (run in order, or selectively)
1. **Raw + Dataset** (or reuse)
2. **Features** (or reuse / shared-cache restore)
3. **Baselines** (or reuse artifacts)
4. **Eval** (slice + stability reports)
5. **Export** (summary, UI bundle, handover, evidence)

## Deep dives (optional)
- `02_featurelab_login_attempt.ipynb` — feature exploration
- `03_baselinelab_login_attempt.ipynb` — baseline exploration

---

In [1]:
from pathlib import Path
import os
import inkswarm_detectlab

# Robust repo root detection (works regardless of Jupyter CWD)
pkg_file = Path(inkswarm_detectlab.__file__).resolve()  # .../src/inkswarm_detectlab/__init__.py
repo_root = pkg_file.parents[2]
if repo_root.name == 'src':
    repo_root = repo_root.parent

# Locate configs dir
configs_dir = repo_root / 'configs'
if not configs_dir.exists():
    # Fallback: walk upwards looking for a configs/ folder
    cand = repo_root
    while cand != cand.parent and not (cand / 'configs').exists():
        cand = cand.parent
    configs_dir = cand / 'configs'

# Choose config file
cfg_basename = os.environ.get('INKSWARM_CFG', 'skynet_smoke.yaml')
matches = list(configs_dir.rglob(cfg_basename)) if configs_dir.exists() else []
if not matches:
    # Fallback: pick any yaml in configs_dir
    matches = sorted(configs_dir.rglob('*.yaml')) if configs_dir.exists() else []
if not matches:
    raise FileNotFoundError(f'Could not find config under configs_dir={configs_dir} (repo_root={repo_root})')

CFG_PATH = matches[0]
print('REPO_ROOT:', repo_root)
print('CONFIGS_DIR:', configs_dir)
print('CFG_PATH:', CFG_PATH)


REPO_ROOT: C:\Users\Martín\Desktop\inkswarm-core\usul-inkswarm-detectlab
CONFIGS_DIR: C:\Users\Martín\Desktop\inkswarm-core\usul-inkswarm-detectlab\configs
CFG_PATH: C:\Users\Martín\Desktop\inkswarm-core\usul-inkswarm-detectlab\configs\skynet_smoke.yaml


In [2]:
# Core imports for the step notebook (kept in one place for restart safety)

import json
from pathlib import Path

from inkswarm_detectlab.config import load_config
from inkswarm_detectlab.ui.steps import StepRecorder
from inkswarm_detectlab.ui.notebook_tools import print_run_tree, tail_text
from inkswarm_detectlab.ui.step_runner import (
    resolve_run_id,
    wire_check,
    step_dataset,
    step_features,
    step_baselines,
    step_eval,
    step_export,
)

print("Imports OK")


Imports OK


In [3]:
# DEFAULT TOGGLES (restart-safe)
# This notebook is designed to be runnable top-down OR resumed mid-way after restart.
# If you run a later cell directly, these defaults prevent NameError for toggles.
#
# You can override any of these variables in your own cell above the steps.

# Run id: set to an existing run to maximize reuse
RUN_ID = globals().get("RUN_ID", None)  # e.g. "RR2_MVP_GIT_B_0002"

# Reuse policy
REUSE_IF_EXISTS = globals().get("REUSE_IF_EXISTS", True)

# Step toggles
DO_STEP_1_DATASET  = globals().get("DO_STEP_1_DATASET", True)
FORCE_STEP_1_DATASET = globals().get("FORCE_STEP_1_DATASET", False)

DO_STEP_2_FEATURES = globals().get("DO_STEP_2_FEATURES", True)
FORCE_STEP_2_FEATURES = globals().get("FORCE_STEP_2_FEATURES", False)
USE_SHARED_FEATURE_CACHE = globals().get("USE_SHARED_FEATURE_CACHE", True)
WRITE_SHARED_FEATURE_CACHE = globals().get("WRITE_SHARED_FEATURE_CACHE", True)

DO_STEP_3_BASELINES = globals().get("DO_STEP_3_BASELINES", True)
FORCE_STEP_3_BASELINES = globals().get("FORCE_STEP_3_BASELINES", False)

DO_STEP_4_EVAL = globals().get("DO_STEP_4_EVAL", True)
FORCE_STEP_4_EVAL = globals().get("FORCE_STEP_4_EVAL", False)

DO_STEP_5_EXPORT = globals().get("DO_STEP_5_EXPORT", True)
FORCE_STEP_5_EXPORT = globals().get("FORCE_STEP_5_EXPORT", False)

print("Toggles ready:",
      "RUN_ID=", RUN_ID,
      "REUSE_IF_EXISTS=", REUSE_IF_EXISTS)


Toggles ready: RUN_ID= None REUSE_IF_EXISTS= True


In [4]:
# PRE-FLIGHT (restart-safe): ensure key symbols exist
# This cell prevents NameError when running cells out-of-order or after a kernel restart.

missing = []

# Ensure core step functions are importable
try:
    from inkswarm_detectlab.ui.step_runner import (
        resolve_run_id,
        wire_check,
        step_dataset,
        step_features,
        step_baselines,
        step_eval,
        step_export,
    )
except Exception as e:
    raise RuntimeError(f"Failed importing step_runner helpers: {e}") from e

# Ensure utilities are importable
try:
    from inkswarm_detectlab.ui.steps import StepRecorder
    from inkswarm_detectlab.ui.notebook_tools import print_run_tree, tail_text
except Exception as e:
    raise RuntimeError(f"Failed importing notebook utilities: {e}") from e

# Ensure cfg/run_id are initialized
from inkswarm_detectlab.config import load_config
if "cfg" not in globals():
    cfg = load_config(CFG_PATH)

if "run_id" not in globals():
    _rid = globals().get("RUN_ID", None)
    cfg, run_id = resolve_run_id(CFG_PATH, run_id=_rid)

print("Pre-flight OK:",
      "run_id=", run_id,
      "cfg_path=", CFG_PATH)


Pre-flight OK: run_id= RUN_SAMPLE_SMOKE_0001 cfg_path= C:\Users\Martín\Desktop\inkswarm-core\usul-inkswarm-detectlab\configs\skynet_smoke.yaml


In [5]:
# Step 0 — wiring / paths check (Dry-run A)
# Self-contained: imports + guards so it works after a fresh kernel restart.

import json
from pathlib import Path

from inkswarm_detectlab.config import load_config
from inkswarm_detectlab.ui.step_runner import wire_check, resolve_run_id

# Ensure cfg + run_id exist
if "cfg" not in globals():
    cfg = load_config(CFG_PATH)

if "run_id" not in globals():
    # Try to honor user-provided RUN_ID if present
    _rid = globals().get("RUN_ID", None)
    cfg, run_id = resolve_run_id(CFG_PATH, run_id=_rid)
else:
    # keep existing run_id variable
    pass

check = wire_check(CFG_PATH, run_id=run_id)
print(json.dumps(check, indent=2))
rdir = Path(check["paths"]["run_dir"])

print("\nRun tree (quick):")
from inkswarm_detectlab.ui.notebook_tools import print_run_tree
print_run_tree(rdir)


{
  "run_id": "RUN_SAMPLE_SMOKE_0001",
  "paths": {
    "run_dir": "runs\\RUN_SAMPLE_SMOKE_0001",
    "raw_login_attempt": "runs\\RUN_SAMPLE_SMOKE_0001\\raw\\login_attempt.parquet",
    "raw_checkout_attempt": "runs\\RUN_SAMPLE_SMOKE_0001\\raw\\checkout_attempt.parquet",
    "dataset_train": "runs\\RUN_SAMPLE_SMOKE_0001\\dataset\\login_attempt\\train.parquet",
    "dataset_time_eval": "runs\\RUN_SAMPLE_SMOKE_0001\\dataset\\login_attempt\\time_eval.parquet",
    "dataset_user_holdout": "runs\\RUN_SAMPLE_SMOKE_0001\\dataset\\login_attempt\\user_holdout.parquet",
    "features_login": "runs\\RUN_SAMPLE_SMOKE_0001\\features\\login_attempt\\features.parquet",
    "baselines_dir": "runs\\RUN_SAMPLE_SMOKE_0001\\models\\login_attempt\\baselines",
    "eval_reports_dir": "runs\\RUN_SAMPLE_SMOKE_0001\\reports",
    "share_dir": "runs\\RUN_SAMPLE_SMOKE_0001\\share",
    "manifest": "runs\\RUN_SAMPLE_SMOKE_0001\\manifest.json"
  }
}

Run tree (quick):
- share/logs: runs\RUN_SAMPLE_SMOKE_0001\share

In [6]:
# Utility for consistent step prints
def _print_outcome(out):
    print(f"\n=== STEP: {out.name} ===")
    print("status:", out.status)
    print("decision:", f"{out.decision.mode} — {out.decision.reason}")
    print("decision_meta:", f"used_manifest={out.decision.used_manifest} forced={out.decision.forced}")
    if out.notes:
        print("notes:")
        for n in out.notes:
            print(" -", n)
    if out.outputs:
        print("outputs:")
        for k, a in out.outputs.items():
            try:
                p = a.get("path")  # if dict-like
                ex = a.get("exists")
                print(f" - {k}: {p} (exists={ex})")
            except Exception:
                print(f" - {k}: {getattr(a, 'path', '')} (exists={getattr(a, 'exists', None)})")
    if out.summary:
        print("summary:", out.summary)


In [7]:
from inkswarm_detectlab.ui.step_runner import step_dataset  # guard for out-of-order runs
# Step 1 — Raw + Dataset (or reuse)
rec = StepRecorder()
out1 = None
if DO_STEP_1_DATASET:
    out1 = step_dataset(
        cfg,
        cfg_path=CFG_PATH,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_1_DATASET,
    )
    _print_outcome(out1)
else:
    print("Skipped (DO_STEP_1_DATASET=False)")



=== STEP: dataset ===
status: skipped
decision: reuse — Manifest step record matches current config hash and outputs exist.
decision_meta: used_manifest=True forced=False
notes:
 - Skipped compute based on reuse policy.
outputs:
 - run_dir: runs\RUN_SAMPLE_SMOKE_0001 (exists=True)
 - raw_login_attempt: runs\RUN_SAMPLE_SMOKE_0001\raw\login_attempt.parquet (exists=True)
 - raw_checkout_attempt: runs\RUN_SAMPLE_SMOKE_0001\raw\checkout_attempt.parquet (exists=True)
 - dataset_train: runs\RUN_SAMPLE_SMOKE_0001\dataset\login_attempt\train.parquet (exists=True)
 - dataset_time_eval: runs\RUN_SAMPLE_SMOKE_0001\dataset\login_attempt\time_eval.parquet (exists=True)
 - dataset_user_holdout: runs\RUN_SAMPLE_SMOKE_0001\dataset\login_attempt\user_holdout.parquet (exists=True)


In [8]:
from pathlib import Path
# Step 1b — Labels + sanity checks (lightweight)
# Surfaces label assignment logic (non-random) and basic distributions.
from inkswarm_detectlab.synthetic.label_defs import as_markdown_table
from inkswarm_detectlab.io.tables import read_auto

rdir = Path(cfg.paths.runs_dir) / run_id  # cfg.paths.runs_dir is already runs_dir

print("\nLabel definitions (synthetic scenarios):")
print(as_markdown_table())

train_path = (rdir / "dataset" / "login_attempt" / "train.parquet")
if train_path.exists():
    df = read_auto(train_path)
    label_cols = [c for c in df.columns if c.startswith("label_")]
    print("\nTrain split size:", len(df))
    if label_cols:
        print("Label prevalence (train):")
        for c in sorted(label_cols):
            prev = float(df[c].mean())
            print(f" - {c}: {prev:.4f}")
    else:
        print("No label_ columns found in dataset table (unexpected).")
else:
    print("Train split missing; run Step 1 first.")



Label definitions (synthetic scenarios):
| label key | title | intent | how generated |
|---|---|---|---|
| `label_replicators` | Replicators | Automated or semi-automated high-volume login attempts (spray / scripted). | Assigned in the synthetic generator based on a per-row 'is_attacker' flag and an attack-campaign intensity bucket; deterministic given the generator seed. |
| `label_the_mule` | The Mule | Credential-stuffing / account takeover style attempts with stronger success bias. | Assigned in the synthetic generator from the same attacker campaign scaffold; deterministic given the generator seed. |
| `label_the_chameleon` | The Chameleon | Evasive attacker behavior designed to blend into normal traffic. | Assigned from the attacker campaign scaffold with higher 'stealth' characteristics; deterministic given the generator seed. |

Train split size: 8142
Label prevalence (train):
 - label_benign: 0.9442
 - label_replicators: 0.0257
 - label_the_chameleon: 0.0183
 - label_the_mul

In [9]:
# Step 2 — Features (or reuse / shared-cache restore)
out2 = None
if DO_STEP_2_FEATURES:
    out2 = step_features(
        cfg,
        cfg_path=CFG_PATH,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_2_FEATURES,
        use_cache=USE_SHARED_FEATURE_CACHE,
        write_cache=WRITE_SHARED_FEATURE_CACHE,
    )
    _print_outcome(out2)
    print("\nLog tail (featurelab):")
    log_path = Path(cfg.paths.runs_dir) / run_id / "share" / "logs" / "featurelab.log"
    if log_path.exists():
        print(tail_text(log_path, n_lines=120))
    else:
        print("(no featurelab.log yet)")
else:
    print("Skipped (DO_STEP_2_FEATURES=False)")



=== STEP: features ===
status: skipped
decision: reuse — Manifest step record matches current config hash and outputs exist.
decision_meta: used_manifest=True forced=False
notes:
 - Skipped build based on reuse policy.
outputs:
 - features_login: runs\RUN_SAMPLE_SMOKE_0001\features\login_attempt\features.parquet (exists=True)

Log tail (featurelab):
(no featurelab.log yet)


In [10]:
# Step 3 — Baselines (or reuse artifacts)
out3 = None
if DO_STEP_3_BASELINES:
    out3 = step_baselines(
        cfg,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_3_BASELINES,
        cfg_path=CFG_PATH,
    )
    _print_outcome(out3)
    # Quick look at metrics.json if present
    metrics_path = Path(cfg.paths.runs_dir) / run_id / "models" / "login_attempt" / "baselines" / "metrics.json"
    if metrics_path.exists():
        metrics = json.loads(metrics_path.read_text(encoding="utf-8"))
        print("\nBaseline summary (meta):")
        print(json.dumps(metrics.get("meta", {}), indent=2))
else:
    print("Skipped (DO_STEP_3_BASELINES=False)")



=== STEP: baselines ===
status: skipped
decision: reuse — Manifest step record matches current config hash and outputs exist.
decision_meta: used_manifest=True forced=False
notes:
 - Skipped training based on reuse policy.
outputs:
 - baselines_dir: runs\RUN_SAMPLE_SMOKE_0001\models\login_attempt\baselines (exists=True)
 - metrics_json: runs\RUN_SAMPLE_SMOKE_0001\models\login_attempt\baselines\metrics.json (exists=True)
summary: {'env': {'pandas': '2.3.3', 'platform': 'Windows-10-10.0.19045-SP0', 'pyarrow': '22.0.0', 'python': '3.14.0 (tags/v3.14.0:ebf955d, Oct  7 2025, 10:15:03) [MSC v.1944 64 bit (AMD64)]', 'sklearn': '1.8.0', 'threadpools': [{'architecture': 'Haswell', 'filepath': 'C:\\Users\\Martín\\AppData\\Local\\Programs\\Python\\Python314\\Lib\\site-packages\\numpy.libs\\libscipy_openblas64_-9e3e5a4229c1ca39f10dc82bba9e2b2b.dll', 'internal_api': 'openblas', 'num_threads': 8, 'prefix': 'libscipy_openblas', 'threading_layer': 'pthreads', 'user_api': 'blas', 'version': '0.3.30'},

In [11]:
# Step 4 — Eval (slice + stability reports)
out4 = None
if DO_STEP_4_EVAL:
    out4 = step_eval(
        cfg,
        cfg_path=CFG_PATH,
        run_id=run_id,
        rec=rec,
        reuse_if_exists=REUSE_IF_EXISTS,
        force=FORCE_STEP_4_EVAL,
    )
    _print_outcome(out4)
    # Point to key report files
    rdir = Path(cfg.paths.runs_dir) / run_id
    for rel in [
        "reports/eval_slices_login_attempt.md",
        "reports/eval_stability_login_attempt.md",
    ]:
        p = rdir / rel
        print(f" - {rel}: {'OK' if p.exists() else 'missing'} ({p})")
else:
    print("Skipped (DO_STEP_4_EVAL=False)")



=== STEP: eval ===
status: skipped
decision: reuse — Manifest step record matches current config hash and outputs exist.
decision_meta: used_manifest=True forced=False
notes:
 - Skipped eval based on reuse policy.
outputs:
 - reports_dir: runs\RUN_SAMPLE_SMOKE_0001\reports (exists=True)
 - eval_slices_json: runs\RUN_SAMPLE_SMOKE_0001\reports\eval_slices_login_attempt.json (exists=True)
 - eval_stability_json: runs\RUN_SAMPLE_SMOKE_0001\reports\eval_stability_login_attempt.json (exists=True)
 - eval_slices_md: runs\RUN_SAMPLE_SMOKE_0001\reports\eval_slices_login_attempt.md (exists=True)
 - eval_stability_md: runs\RUN_SAMPLE_SMOKE_0001\reports\eval_stability_login_attempt.md (exists=True)
 - reports/eval_slices_login_attempt.md: OK (runs\RUN_SAMPLE_SMOKE_0001\reports\eval_slices_login_attempt.md)
 - reports/eval_stability_login_attempt.md: OK (runs\RUN_SAMPLE_SMOKE_0001\reports\eval_stability_login_attempt.md)


In [12]:
from pathlib import Path
import inkswarm_detectlab.ui.step_runner as sr
import inspect

rdir = Path(cfg.paths.runs_dir) / run_id
share_dir = rdir / "share"
out_dir = share_dir / "reports"   # most consistent with the tree your notebook expects
out_dir.mkdir(parents=True, exist_ok=True)

print("export_ui_bundle signature:", inspect.signature(sr.export_ui_bundle))

sr.export_ui_bundle(cfg, run_ids=[run_id], out_dir=out_dir)
from pathlib import Path
import json
import inkswarm_detectlab.ui.step_runner as sr

runs_dir = Path(cfg.paths.runs_dir)
rdir = runs_dir / run_id
share_dir = rdir / "share"
out_dir = share_dir / "reports"
out_dir.mkdir(parents=True, exist_ok=True)

# 1) Write UI summary (so we have something to feed into handover)
sr.write_ui_summary(cfg, run_id=run_id)

# 2) Locate the summary artifact (try common locations)
summary_path_candidates = [
    share_dir / "summary.json",
    share_dir / "summary.md",  # not json, but keep as fallback
    rdir / "reports" / "summary.json",
]
summary = None

for p in summary_path_candidates:
    if p.exists() and p.suffix.lower() == ".json":
        summary = json.loads(p.read_text(encoding="utf-8"))
        print("Loaded summary from:", p)
        break

if summary is None:
    # fallback: minimal summary dict
    summary = {"run_id": run_id, "note": "summary.json not found; using minimal summary"}

# 3) Build UI bundle and capture where it went
ui_bundle_dir = sr.export_ui_bundle(cfg, run_ids=[run_id], out_dir=out_dir)
print("ui_bundle_dir:", ui_bundle_dir)

# 4) Exec summary (already working for you)
sr.write_exec_summary(run_dir=rdir)

# 5) Handover (NOW with required keyword-only args)
handover_path = sr.write_mvp_handover(
    runs_dir=runs_dir,
    run_id=run_id,
    ui_bundle_dir=Path(ui_bundle_dir) if ui_bundle_dir is not None else None,
    summary=summary,
)
print("handover_path:", handover_path)

# 6) Tree
from inkswarm_detectlab.ui.notebook_tools import print_run_tree
print_run_tree(rdir)


from inkswarm_detectlab.ui.notebook_tools import print_run_tree
print_run_tree(rdir)



export_ui_bundle signature: (cfg: 'AppConfig', *, run_ids: 'list[str]', out_dir: 'Path', force: 'bool' = False) -> 'Path'
ui_bundle_dir: runs\RUN_SAMPLE_SMOKE_0001\share\reports
handover_path: runs\RUN_SAMPLE_SMOKE_0001\reports\mvp_handover.md
- share/logs: runs\RUN_SAMPLE_SMOKE_0001\share\logs (missing)
- share/reports: runs\RUN_SAMPLE_SMOKE_0001\share\reports 
- models: runs\RUN_SAMPLE_SMOKE_0001\models 
- reports: runs\RUN_SAMPLE_SMOKE_0001\reports 
- logs: runs\RUN_SAMPLE_SMOKE_0001\logs 
- share/logs: runs\RUN_SAMPLE_SMOKE_0001\share\logs (missing)
- share/reports: runs\RUN_SAMPLE_SMOKE_0001\share\reports 
- models: runs\RUN_SAMPLE_SMOKE_0001\models 
- reports: runs\RUN_SAMPLE_SMOKE_0001\reports 
- logs: runs\RUN_SAMPLE_SMOKE_0001\logs 


In [13]:
import inspect
print("write_mvp_handover signature:", inspect.signature(sr.write_mvp_handover))
print("defined in:", inspect.getsourcefile(sr.write_mvp_handover))


write_mvp_handover signature: (*, runs_dir: 'Path', run_id: 'str', ui_bundle_dir: 'Path | None', summary: 'dict[str, Any]') -> 'Path'
defined in: C:\Users\Martín\Desktop\inkswarm-core\usul-inkswarm-detectlab\src\inkswarm_detectlab\mvp\handover.py


In [14]:
# ==========================
# Step 5 — Export share bundle (manual, robust)
# ==========================
# Goals:
# - Avoid patches by calling the current functions with their actual signatures.
# - Produce: UI summary, exec summary, UI bundle, handover, (optional) evidence bundle.
# - Be restart-safe and print where outputs landed.

from __future__ import annotations

import json
from pathlib import Path
import inspect

import inkswarm_detectlab.ui.step_runner as sr
from inkswarm_detectlab.ui.notebook_tools import print_run_tree


# --------------------------
# 0) Resolve key paths
# --------------------------
runs_dir = Path(cfg.paths.runs_dir)
rdir = runs_dir / run_id
share_dir = rdir / "share"
share_dir.mkdir(parents=True, exist_ok=True)

# Where to write bundle assets (your export_ui_bundle requires out_dir)
ui_out_dir = share_dir / "reports"
ui_out_dir.mkdir(parents=True, exist_ok=True)

print("Step 5 paths:")
print(" - runs_dir:", runs_dir)
print(" - run_dir :", rdir)
print(" - share_dir:", share_dir)
print(" - ui_out_dir:", ui_out_dir)


# --------------------------
# 1) UI Summary
# --------------------------
print("\n[5A] write_ui_summary signature:", inspect.signature(sr.write_ui_summary))
sr.write_ui_summary(cfg, run_id=run_id)

# Try to load a JSON summary if it exists (handover needs a dict summary)
summary_candidates = [
    share_dir / "summary.json",
    share_dir / "ui_summary.json",
    rdir / "reports" / "summary.json",
    rdir / "share" / "summary.json",
]

summary: dict = {"run_id": run_id, "note": "summary.json not found; using minimal summary dict"}
for p in summary_candidates:
    if p.exists() and p.suffix.lower() == ".json":
        try:
            summary = json.loads(p.read_text(encoding="utf-8"))
            print("[5A] Loaded summary JSON:", p)
            break
        except Exception as e:
            print("[5A] Failed reading summary JSON:", p, "err=", e)

print("[5A] summary keys:", sorted(list(summary.keys()))[:25], ("..." if len(summary.keys()) > 25 else ""))


# --------------------------
# 2) Exec Summary (keyword-only)
# --------------------------
print("\n[5B] write_exec_summary signature:", inspect.signature(sr.write_exec_summary))
# Current signature (confirmed): (*, run_dir: Path, rr_provisional=True, d0004_deferred=True)
exec_art = sr.write_exec_summary(run_dir=rdir)
print("[5B] exec_summary artifacts:", exec_art)


# --------------------------
# 3) UI Bundle (keyword-only run_ids + out_dir)
# --------------------------
print("\n[5C] export_ui_bundle signature:", inspect.signature(sr.export_ui_bundle))
# Current signature (confirmed): (cfg, *, run_ids: list[str], out_dir: Path, force=False) -> Path
ui_bundle_dir = sr.export_ui_bundle(cfg, run_ids=[run_id], out_dir=ui_out_dir)
print("[5C] ui_bundle_dir:", ui_bundle_dir)


# --------------------------
# 4) Handover (keyword-only, requires runs_dir + ui_bundle_dir + summary dict)
# --------------------------
print("\n[5D] write_mvp_handover signature:", inspect.signature(sr.write_mvp_handover))
# Current signature (confirmed):
#   (*, runs_dir: Path, run_id: str, ui_bundle_dir: Path|None, summary: dict[str, Any]) -> Path
handover_path = sr.write_mvp_handover(
    runs_dir=runs_dir,
    run_id=run_id,
    ui_bundle_dir=Path(ui_bundle_dir) if ui_bundle_dir is not None else None,
    summary=summary,
)
print("[5D] handover_path:", handover_path)


# --------------------------
# 5) Evidence bundle (optional)
# --------------------------
print("\n[5E] evidence bundle (optional)")
try:
    from inkswarm_detectlab.share.evidence import export_evidence_bundle
    export_evidence_bundle(cfg, run_id=run_id)
    print("[5E] evidence bundle: OK")
except Exception as e:
    print("[5E] evidence bundle skipped:", e)


# --------------------------
# 6) Final tree
# --------------------------
print("\nRun tree (after Step 5 manual export):")
print_run_tree(rdir)


Step 5 paths:
 - runs_dir: runs
 - run_dir : runs\RUN_SAMPLE_SMOKE_0001
 - share_dir: runs\RUN_SAMPLE_SMOKE_0001\share
 - ui_out_dir: runs\RUN_SAMPLE_SMOKE_0001\share\reports

[5A] write_ui_summary signature: (cfg: 'AppConfig', *, run_id: 'str', force: 'bool' = False) -> 'Path'
[5A] summary keys: ['note', 'run_id'] 

[5B] write_exec_summary signature: (*, run_dir: 'Path', rr_provisional: 'bool' = True, d0004_deferred: 'bool' = True) -> 'ExecSummaryArtifacts'
[5B] exec_summary artifacts: ExecSummaryArtifacts(exec_md=WindowsPath('runs/RUN_SAMPLE_SMOKE_0001/reports/EXEC_SUMMARY.md'), exec_html=WindowsPath('runs/RUN_SAMPLE_SMOKE_0001/reports/EXEC_SUMMARY.html'), summary_md=WindowsPath('runs/RUN_SAMPLE_SMOKE_0001/reports/summary.md'), summary_html=WindowsPath('runs/RUN_SAMPLE_SMOKE_0001/reports/summary.html'))

[5C] export_ui_bundle signature: (cfg: 'AppConfig', *, run_ids: 'list[str]', out_dir: 'Path', force: 'bool' = False) -> 'Path'
[5C] ui_bundle_dir: runs\RUN_SAMPLE_SMOKE_0001\share\re

In [15]:
# Step summary table (D-0022 visibility)
print(rec.to_markdown())


### Step summary

(no steps recorded)

